# API Calls

## Data

In [4]:
import os

folder = "/Users/fanch/Downloads/Seminaire2018"

def get_dir_contents(dirPath):
    listFiles = []
    listDirs = []
    for root, dirs, files in os.walk(dirPath):
        for f in files:
            listFiles.append(os.path.join(root, f))
        for d in dirs:
            listDirs.append(os.path.join(root, d))
    return listFiles, listDirs

listFiles, _ = get_dir_contents(folder)
listFilesAPI = [open(filename, 'rb') for filename in listFiles]
len(listFiles)

125

## One shot call

In [20]:
import requests
url = "http://127.0.0.1/get-embedding/"
file = {'file' : (listFiles[1], open(listFiles[1], 'rb'), "image/jpeg")}
response = requests.post(url, files=file).json()

In [21]:
print(response["filename"])
print(response["embedding"][:10])

/Users/fanch/Downloads/Seminaire2018/7N9A2919.jpg
[0.0, 0.05359392985701561, 0.0, 0.03197852522134781, 0.0007716820691712201, 0.006001245696097612, 0.05950940027832985, 0.01724093407392502, 0.0026352705899626017, 0.0026091327890753746]


## One shot call with normalize = False

In [7]:
url = "http://127.0.0.1/get-embedding/"
file = {'file' : (listFiles[1], open(listFiles[1], 'rb'), "image/jpeg")}
response = requests.post(url, files=file, params = {"normalize":"False"}).json()

In [8]:
print(response["filename"])
print(response["embedding"][:10])

/Users/fanch/Downloads/Seminaire2018/7N9A2919.jpg
[0.0, 1.3261535167694092, 0.0, 0.7912917733192444, 0.019094865769147873, 0.14849765598773956, 1.4725286960601807, 0.4266178011894226, 0.06520838290452957, 0.06456161290407181]


## Loop of one shot calls

In [9]:
url = "http://127.0.0.1/get-embedding/"
result_list = []
for filename in listFiles:
    file = {'file' : (filename, open(filename, 'rb'), "image/jpeg")} 
    response = requests.post(url, files=file).json()
    result_list.append(response)
    
response = {"embedding_list":result_list}

In [10]:
print(response["embedding_list"][1]["filename"])
print(response["embedding_list"][1]["embedding"][:10])

/Users/fanch/Downloads/Seminaire2018/7N9A2919.jpg
[0.0, 0.05359392985701561, 0.0, 0.03197852522134781, 0.0007716820691712201, 0.006001245696097612, 0.05950940027832985, 0.01724093407392502, 0.0026352705899626017, 0.0026091327890753746]


## Give file list

In [13]:
url_list = "http://127.0.0.1/get-embedding-from-list/"
listFilesAPI = [("files", (filename, open(filename, 'rb'))) for filename in listFiles]
r = requests.post(url_list, files=listFilesAPI)
response = r.json()

In [14]:
print(response["embedding_list"][1]["filename"])
print(response["embedding_list"][1]["embedding"][:10])

/Users/fanch/Downloads/Seminaire2018/7N9A2919.jpg
[0.0, 0.053593941032886505, 0.0, 0.0319785550236702, 0.0007716873078607023, 0.006001209374517202, 0.059509385377168655, 0.017240941524505615, 0.002635287120938301, 0.0026091488543897867]


## Give file list with normalize = False

In [15]:
url_list = "http://127.0.0.1/get-embedding-from-list/"
listFilesAPI = [("files", (filename, open(filename, 'rb'))) for filename in listFiles]
r = requests.post(url_list, files=listFilesAPI, params = {"normalize":"False"})
response = r.json()

In [16]:
print(response["embedding_list"][1]["filename"])
print(response["embedding_list"][1]["embedding"][:10])

/Users/fanch/Downloads/Seminaire2018/7N9A2919.jpg
[0.0, 1.3261536359786987, 0.0, 0.7912923097610474, 0.01909499242901802, 0.14849673211574554, 1.472528100013733, 0.42661792039871216, 0.06520877778530121, 0.06456200033426285]


## Try list of lists

In [19]:
def break_lst(lst, size):
    if size>len(lst):
        return lst
    else:
        result = []
        num_sublists = int(len(lst)/float(size))
        i=1
        while i<=num_sublists+1:
            if len(lst[(i-1)*size:min(i*size,len(lst))])>0:
                result.append(lst[(i-1)*size:min(i*size,len(lst))])
            i=i+1
        return result

url_list = "http://127.0.0.1/get-embedding-from-list/"
# Give list of list
listFilesAPI = [("files", (filename, open(filename, 'rb'))) for filename in listFiles]
list_of_list = break_lst(listFilesAPI,20)

for lst in list_of_list:
    r = requests.post(url_list,files=lst)
    response = r.json()

# Multiproc or requests

In [17]:
import requests
import multiprocessing

BASE_URI = "http://127.0.0.1/get-embedding/"

def internet_resource_getter(stuff_to_get):
    #session = requests.Session()
    image_file_descriptor = open(stuff_to_get, 'rb')
    files = {'file': image_file_descriptor}
    response = requests.post(url, files=files)
    return response.json()
  
pool = multiprocessing.Pool(processes=8)
pool_outputs = pool.map(internet_resource_getter, listFiles)
pool.close()
pool.join()

In [18]:
response = {"embedding_list" : pool_outputs}
print(response["embedding_list"][1]["filename"])
print(response["embedding_list"][1]["embedding"][:10])

7N9A2919.jpg
[0.0, 0.05359392985701561, 0.0, 0.03197852522134781, 0.0007716820691712201, 0.006001245696097612, 0.05950940027832985, 0.01724093407392502, 0.0026352705899626017, 0.0026091327890753746]
